In [1]:
# Import packages
import pandas as pd
import numpy as np
import pyodbc
import warnings
from datetime import datetime
from datetime import timedelta

warnings.filterwarnings("ignore")

In [2]:
query = f'''
        select *

	from

		((select 
			c.intervalStart, 
			avg(c.Connexus_kWh) Connexus_kWh
		
			from (select 
					dateadd(hour, -1, dateTime) intervalStart, 
					kwh Connexus_kWh
			
				from [GRE_EDW].[cnx_custom].[GREMVWebData]

				union all

				select 
					intervalStart, 
					averageValue * 1000 Connexus_kWh

				from SCADA_EDWSTG.cnx_custom.scadaChronusTimeseriesAggregate 

				where 
					displayKey = '04150001' and 
					intervalStart >= (select max(datetime) from [GRE_EDW].[cnx_custom].[GREMVWebData])) c

		
		group by c.intervalStart) t1

		inner join 
		
		(select *
		from WEATHER_EDW.dbo.vw_weatherHistory
		where station = 'STF ') t2																																													
		
		on t1.intervalStart = t2.dateTime)

	order by intervalStart asc'''

with pyodbc.connect('dsn=WEATHER_EDW') as conn:
            
            load_weather_df = pd.read_sql(query, conn)
            
# Query GRE load management program history

query = f'''
        SELECT
	convert(date, eventStartTime) date
	,[eventStartTime] ProgramStart
	,[eventEndTime] ProgramEnd
	,[DRProgramDescription] Program
FROM [PTR_EDW].[cnx_custom].[demandResponseEventGRE]'''

with pyodbc.connect('dsn=WEATHER_EDW') as conn:
            
            GRE_programs_df = pd.read_sql(query, conn)
            
# Query CNX load management program history

query = f'''
        SELECT
	convert(date, ProgramStart) date
	,[PSOScheduleID]
	,[Program]
	,[ProgramStart]
	,[ProgramEnd]
	,[Notes]
	,[createDate]
	,[updateDate]
	FROM [EDW].[PSO].[tPSOProgramSchedule]'''

with pyodbc.connect('dsn=WEATHER_EDW') as conn:
            
            CNX_programs_df = pd.read_sql(query, conn)

In [3]:
# transform daily program history data to hourly data
def expand_program_df(sample):


    program_name = []
    program_start_datetime = []
    program_duration_mins = []

    running_length = sample['ProgramEnd'].hour -  sample['ProgramStart'].hour + 1

    for i in range(running_length):

        program_name.append(sample['Program'])
        program_start_datetime.append(datetime.strptime(str(sample['ProgramStart']), "%Y-%m-%d %H:%M:%S").replace(second=0, microsecond=0, minute=0) + timedelta(hours=i))

        if i == 0:
            program_first_mins = 60 - datetime.strptime(str(sample['ProgramStart']), "%Y-%m-%d %H:%M:%S").minute
            program_duration_mins.append(program_first_mins)
            
        
        elif i != running_length-1:
            program_duration_mins.append(60)
        
        else:
            program_end_mins = datetime.strptime(str(sample['ProgramEnd']), "%Y-%m-%d %H:%M:%S").minute - 0
            program_duration_mins.append(program_end_mins)
    
    return {'program_name':program_name, 'program_start_datetime':program_start_datetime, 'program_duration_mins':program_duration_mins}

    

In [4]:
# modification process
if len(CNX_programs_df)!=0:

    expanded_CNX_program_df = pd.DataFrame()
    for i in range(len(CNX_programs_df)):
        expanded_CNX_program_df = pd.concat([expanded_CNX_program_df, pd.DataFrame(expand_program_df(CNX_programs_df.iloc[i]))], axis=0)

    expanded_CNX_program_df = expanded_CNX_program_df.rename( \
        columns = {'program_start_datetime':'intervalStart'
                ,'program_name':'CNX_program_name' \
                    ,'program_duration_mins':'CNX_program_duration_mins'})

    for index, i in enumerate(expanded_CNX_program_df['CNX_program_name'].unique()):
        
        if index == 0:
            CNX_all_program_df = expanded_CNX_program_df[expanded_CNX_program_df['CNX_program_name']==i]
            CNX_all_program_df =CNX_all_program_df.rename(columns={'CNX_program_duration_mins':f'{i}_duration_mins'})
            CNX_all_program_df = CNX_all_program_df.drop(columns=['CNX_program_name'])

        else:
            CNX_one_program_df = expanded_CNX_program_df[expanded_CNX_program_df['CNX_program_name']==i]
            CNX_one_program_df = CNX_one_program_df.rename(columns={'CNX_program_duration_mins':f'{i}_duration_mins'})
            CNX_one_program_df = CNX_one_program_df.drop(columns=['CNX_program_name'])

            CNX_all_program_df = CNX_all_program_df.merge(CNX_one_program_df, on='intervalStart', how='outer')

In [5]:
# modification process
expanded_GRE_program_df = pd.DataFrame()
for i in range(len(GRE_programs_df)):
    expanded_GRE_program_df = pd.concat([expanded_GRE_program_df, pd.DataFrame(expand_program_df(GRE_programs_df.iloc[i]))], axis=0)

    
expanded_GRE_program_df = expanded_GRE_program_df.rename( \
    columns = {'program_start_datetime':'intervalStart'
               ,'program_name':'GRE_program_name' \
                ,'program_duration_mins':'GRE_program_duration_mins'})

for index, i in enumerate(expanded_GRE_program_df['GRE_program_name'].unique()):
    
    if index == 0:
        GRE_all_program_df = expanded_GRE_program_df[expanded_GRE_program_df['GRE_program_name']==i]
        GRE_all_program_df =GRE_all_program_df.rename(columns={'GRE_program_duration_mins':f'{i}_duration_mins'})
        GRE_all_program_df = GRE_all_program_df.drop(columns=['GRE_program_name'])

    else:
        GRE_one_program_df = expanded_GRE_program_df[expanded_GRE_program_df['GRE_program_name']==i]
        GRE_one_program_df = GRE_one_program_df.rename(columns={'GRE_program_duration_mins':f'{i}_duration_mins'})
        GRE_one_program_df = GRE_one_program_df.drop(columns=['GRE_program_name'])

        GRE_all_program_df = GRE_all_program_df.merge(GRE_one_program_df, on='intervalStart', how='outer')

GRE_all_program_df = GRE_all_program_df.filter(['intervalStart', 'Interruptible Irrigation_duration_mins', 'Cycled Air Conditioning_duration_mins', 'Interruptible Water Heating_duration_mins'])

In [6]:
merge_1 = load_weather_df.merge(CNX_all_program_df, on='intervalStart', how='left')
load_weather_programs_df = merge_1.merge(GRE_all_program_df, on='intervalStart', how='left')

In [7]:
load_weather_programs_df = load_weather_programs_df.drop(['dateTime'], axis=1)

In [8]:
load_weather_programs_df[['precipAccumulation', 'precipIntensity', 'precipProbability', \
                          'snowAccumulation', 'snowIntensity']] = \
                          load_weather_programs_df[['precipAccumulation', 'precipIntensity', 'precipProbability', \
                          'snowAccumulation', 'snowIntensity']].fillna(value=0.00)

In [9]:
load_weather_programs_df['precipType'] = load_weather_programs_df['precipType'].fillna(value='no_precip')

In [10]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

In [11]:
imp_nan = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
imp_none = SimpleImputer(missing_values=None, strategy='most_frequent')

In [12]:
load_weather_programs_df['cloudCover'] = imp_nan.fit_transform(load_weather_programs_df[['cloudCover']]).ravel()

load_weather_programs_df['icon'] = imp_nan.fit_transform(load_weather_programs_df[['icon']]).ravel()
load_weather_programs_df['icon'] = imp_none.fit_transform(load_weather_programs_df[['icon']]).ravel()


load_weather_programs_df['uvIndex'] = imp_nan.fit_transform(load_weather_programs_df[['uvIndex']]).ravel()

load_weather_programs_df['visibility'] = imp_nan.fit_transform(load_weather_programs_df[['visibility']]).ravel()

load_weather_programs_df['windGust'] = imp_nan.fit_transform(load_weather_programs_df[['windGust']]).ravel()


In [13]:
for program in load_weather_programs_df.columns[24:]:

    load_weather_programs_df[program] = load_weather_programs_df[program].fillna(value=0)

In [14]:
category_columns = load_weather_programs_df.describe(include=object).columns

In [15]:
le = LabelEncoder()

for one_column in category_columns:

    label = le.fit_transform(load_weather_programs_df[one_column])

    load_weather_programs_df[one_column] = label

In [16]:
knn_imputer = KNNImputer(n_neighbors=5)

In [17]:
rest_columns = load_weather_programs_df.drop(['intervalStart'], axis=1).columns

load_weather_programs_df[rest_columns] = \
    knn_imputer.fit_transform(load_weather_programs_df.drop(['intervalStart'], axis=1))

In [31]:
# load data
df = load_weather_programs_df
# df = df.drop(['Unnamed: 0'], axis=1)
df = df.set_index('intervalStart')


df_copy = df[df.index>='2018'].drop(columns = ['latitude', 'longitude', 'station']).copy(deep=True)

#%%
# Delete duplicates
idx = np.unique(df_copy.index, return_index=True)[1]
df_copy = df_copy.iloc[idx]


def create_features(df):

  temp_df = df.copy(deep=True)

  temp_df['hour'] = temp_df.index.hour
  temp_df['dayofweek'] = temp_df.index.dayofweek
  temp_df['quarter'] = temp_df.index.quarter
  temp_df['month'] = temp_df.index.month
  temp_df['year'] = temp_df.index.year
  temp_df['dayofyear'] = temp_df.index.dayofyear
  temp_df['weekofyear'] = temp_df.index.weekofyear
  temp_df['dayofmonth'] = temp_df.index.day
  temp_df['weekofyear'] = temp_df.index.isocalendar().week

  return temp_df

df_modified = create_features(df_copy)

#%%
# Delete hours that programs were running and programs related features for training set (year < 2023)
def add_time_programs(sample):

    sum_times = 0

    column_list = ['DVR_duration_mins', 'CampusGen_duration_mins',
       'CIGen_duration_mins', 'ACST_duration_mins', 'PTR_duration_mins',
       'Interruptible Irrigation_duration_mins',
       'Cycled Air Conditioning_duration_mins',
       'Interruptible Water Heating_duration_mins']

    for column in column_list:
        try:
          sum_times += sample[column]
        except:
           pass

    sample['sum_time_programs'] = sum_times

    return sample


df_total = df_modified.apply(lambda x: add_time_programs(x), axis=1).reset_index(drop=False)

count = 0
indices_for_delete_hours = []

for i in df_total.index:

    if df_total.iloc[i]['sum_time_programs']!=0:
        count+=1
        indices_for_delete_hours.append(df_total.iloc[i]['intervalStart'])

indices_for_delete_date = []
for i in indices_for_delete_hours:
  indices_for_delete_date.append(i.date())


for i in df_total.columns:
  if '_mins' in i:
    df_total = df_total.drop(columns = [i])


#%%
indices_for_delete_hours

#%%
indices_for_delete_date

#%%
df_total_no_programs_hours = df_total.loc[~df_total['intervalStart'].isin(indices_for_delete_hours)].\
set_index('intervalStart').drop(columns=['sum_time_programs'])

In [70]:
df_total_no_programs_hours.to_csv(r'H:\Power Supply\Interns\2023\CNX load forecasting\datasets\no_programs\train_no_programs_hours.csv')

,Connexus_kWh,unixTime,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipAccumulation,precipIntensity,precipProbability,...,windGust,windSpeed,hour,dayofweek,quarter,month,year,dayofyear,weekofyear,dayofmonth
intervalStart,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00,249481.5859,1514786400.0,-20.92,0.0,-17.87,0.75,21.0,0.0,0.0,0.0,...,6.26,3.11,0.0,0.0,1.0,1.0,2018.0,1.0,1.0,1.0
2018-01-01 01:00:00,238729.8672,1514790000.0,-12.72,0.0,-18.11,0.76,21.0,0.0,0.0,0.0,...,5.18,2.51,1.0,0.0,1.0,1.0,2018.0,1.0,1.0,1.0
2018-01-01 02:00:00,230948.8477,1514793600.0,-13.4,0.0,-18.54,0.77,21.0,0.0,0.0,0.0,...,4.02,1.86,2.0,0.0,1.0,1.0,2018.0,1.0,1.0,1.0
2018-01-01 03:00:00,226906.8906,1514797200.0,-13.68,0.0,-18.72,0.77,21.0,0.0,0.0,0.0,...,4.46,2.21,3.0,0.0,1.0,1.0,2018.0,1.0,1.0,1.0
2018-01-01 04:00:00,226380.1445,1514800800.0,-13.83,0.0,-18.89,0.77,21.0,0.0,0.0,0.0,...,4.7,2.45,4.0,0.0,1.0,1.0,2018.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-21 01:00:00,193443.619193,1703142000.0,24.0,1.0,25.93,0.79,1.0,0.0,0.0,0.0,...,22.65,8.67,1.0,3.0,4.0,12.0,2023.0,355.0,51.0,21.0
2023-12-21 02:00:00,186930.001769,1703145600.0,24.25,0.8,25.93,0.8,1.0,0.0,0.0,0.0,...,21.95,8.25,2.0,3.0,4.0,12.0,2023.0,355.0,51.0,21.0
2023-12-21 03:00:00,185460.942177,1703149200.0,23.98,1.0,26.26,0.81,1.0,0.0,0.0,0.0,...,21.25,8.25,3.0,3.0,4.0,12.0,2023.0,355.0,51.0,21.0
